<a href="https://colab.research.google.com/github/aleklearn/repo/blob/main/generative-ai/lc_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [ ]:
!pip install -Uq langchain openai tiktoken cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
# For Colab, set OPENAI_API_KEY in Secrets then import to environment cariable
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
# If you're running the notebook on your computer, set OPENAI_API_KEY in your .env file
# and install python-dotenv and uncomment code below:

# !pip install

# from dotenv import load_dotenv

# load_dotenv()  # take environment variables from .env

## LLMs & ChatModels


In [ ]:
from langchain.llms import OpenAI

llm = OpenAI()

In [ ]:
text = "What would be a good company name for a company that makes colorful socks?"

llm.predict(text)

'\n\nSocktastic!'

In [ ]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI()

In [ ]:
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"

messages = [HumanMessage(content=text)]

messages

[HumanMessage(content='What would be a good company name for a company that makes colorful socks?')]

In [ ]:
chat_model.predict_messages(messages)

AIMessage(content='ChromaSocks')

## Prompt templates

In [ ]:
from langchain.prompts import PromptTemplate

template = "What is a good name for a company that makes {product}?"

prompt = PromptTemplate.from_template(template)

prompt

PromptTemplate(input_variables=['product'], template='What is a good name for a company that makes {product}?')

In [ ]:
prompt.format(product='colorful socks')

'What is a good name for a company that makes colorful socks?'

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate

system_template = "You are a helpful assistant that translates {input_language} to {output_language}."

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

chat_prompt



ChatPromptTemplate(input_variables=['input_language', 'output_language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], template='You are a helpful assistant that translates {input_language} to {output_language}.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [ ]:
chat_prompt.format_messages(input_language="English",
                            output_language="French",
                            text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

## Chain & Chat Chain

In [ ]:
chain = prompt | llm

chain.invoke({"product": "colorful socks"})

'\n\nSocktastic!'

In [ ]:
chat_chain = chat_prompt | chat_model

chat_chain.invoke({"input_language": "English",
              "output_language": "French",
              "text": "I love programming."})

AIMessage(content="J'adore programmer.")

## Output parsers

In [ ]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
  """Parse the output of an LLM call to a comma-separated list."""

  def parse(self, text: str):
      """Parse the output of an LLM call."""
      return text.strip().split(", ")


In [ ]:
CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

In [ ]:
system_template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that
category in a comma separated list.

ONLY return a comma separated list, and nothing more."""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

chain = chat_prompt | chat_model | CommaSeparatedListOutputParser()

chain.invoke({"text": "colors"})

['red', 'blue', 'green', 'yellow', 'orange']